In [2]:
import pandas as pd
import numpy as np
import sys #Libreria para gestionar directorios 
import os  #
import matplotlib.pyplot as plt
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator #Keras preprocesa las imagenes para entrenar al algoritmos
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras import optimizers #Optimixador para entrenar
from tensorflow.python.keras.models import Sequential #Libreria que permite hacer redes neuronales secuenciales es decir que esta en orden
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation #
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D #Permiten crear las  capas que se utilizan para las convoluciones y pooling
from tensorflow.python.keras import backend as K #Mata procesos dentro de la secion para iniciar uno nuevo
from keras.preprocessing import image
from mlxtend.evaluate import confusion_matrix
import matplotlib.image as mpimg
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

K.clear_session()


#Ubicacion de imagnes en los directorios
data_entrenamiento = './data/entrenamiento'
data_validacion = './data/validacion'

"""
Parameters
"""
epocas=20 # Numero de  veces que se va interar en el entrenamiento
longitud, altura = 150, 150  # Tamano de imagenes que se adaptan  en el procesamiento de imagenes
batch_size = 32 # Numero de imagenes que la compu procesara en cada paso
pasos = 1000 # Nuemero de veces que se va procesar la informacion en cada epoca
validation_steps = 300 #A final de la epocas se van a correr 300 pasos con los datos de validacion para ver que el algitmo esta aprendiendo
filtrosConv1 = 32 #Numeros de filtros
filtrosConv2 = 64 #Numeros de filtros
tamano_filtro1 = (3, 3) #Tamano de filtros
tamano_filtro2 = (2, 2) #Tamano de filtros
tamano_pool = (2, 2) #Tamano de filtro en max pooling
clases = 3 #Numero de clase en el programa gato, perro, y gorilla
lr = 0.0004 #Tasa de aprendizaje, este es el lering ray es que tan grande van hacer los ajustes en la red para acercarse a una solucion optima


##Preparamos nuestras imagenes Preporcesamiento
#Transformacion
#Linea 3 
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255, #Es el rango de pixeles y se convierten de cero a uno para optimzar el entrenamineto y sea rapido
    shear_range=0.2, #Genera las imagnes para aque aprenda en diferente posiciones las inclinas
    zoom_range=0.2, #zoom o partes de una imagen 
    horizontal_flip=True) #Direccionalidad en la imagen invierte la imagen

test_datagen = ImageDataGenerator(rescale=1. / 255)

#Entra al direcrio de entrenaminento y las proceso en funcion a los parametros definidos anteriormente, y clase categorica por los tres animales
entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

#Entra al direcrio de validacion y las proceso en funcion a los parametros definidos anteriormente, y clase categorica por los tres animales
validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

#Creacion de red neuronal convolucional
cnn = Sequential() #varias capas apiladas entre ellas
#Ingreso de parametros en la covolucion, los que va hacer los filtros en las esquinas,  
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
#max pooling
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Flatten())# La imaagen la convierte en una dimencion
cnn.add(Dense(256, activation='relu')) #Capa normar con las 256, neuronas y con funcin relu
cnn.add(Dropout(0.5)) #Parametro para apague el 50 % de capas para que no sabreajuste el aprendizaje
cnn.add(Dense(clases, activation='softmax'))#Ultima capa de sofmas que es la probabilida de la imagen parecida

#Funcino de perdida, optimizacion  y las metricas
cnn.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=lr),
            metrics=['accuracy'])

#Entrenar el algoritmo
cnn.fit_generator(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')#Guarda la estructura del modelo
cnn.save_weights('./modelo/pesos.h5')  #Guarda los pesos de los modelos

Found 193 images belonging to 52 classes.
Found 191 images belonging to 52 classes.
Epoch 1/20


ValueError: A target array with shape (32, 52) was passed for an output of shape (None, 3) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.

In [4]:
N=epocas
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0,N), H.history["Perdida en entrenamiento"], label="Perdida en entrenamiento")
#plt.plot(np.arange(0,N), H.history["Perdida en validacion"], label="Perdida en validacion")
plt.plot(np.arange(0,N), H.history["Precision"], label="Precision en entrenamiento")
#plt.plot(np.arange(0,N), H.history["Precision en validacion"], label="Perdida en entrenamiento")
plt.title("Perdida de entrenamiento y Precision en el set de datos")
plt.xlabel("Epoch #")
plt.ylabel("Perdida/Precision")
plt.legend(loc="loyer left")
plt.savefig("plot.png")

NameError: name 'np' is not defined

<Figure size 432x288 with 0 Axes>

In [5]:
test_datagen= ImageDataGenerator(preprocessing_fution=preprocess_input)

test_generator = test_datagen.flow_from_directory(
    directory="./data/pruebas/",
    target_size=(altura, longitud),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=false,
    seed=42
)

NameError: name 'ImageDataGenerator' is not defined

In [ ]:
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size
test_generator.reset()
pred=cnn.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=1)


In [ ]:
predicted_class_indices= np.argmax(pred,axis=1)

In [ ]:
print(predicted_class_indices)
print(type(predicted_class_indices))

In [ ]:
labels = (imagen_entrenamiento.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]


In [ ]:
filenames= test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results1.csv", index=False)

In [ ]:
real_class_indices=[]
for i in range(0, len(filenames)):
    your_path = filenames[i]
    path_list = your_path.split(os.sep)
    if ("gato" in path_list[1]):
        real_class_indices.append(0)
    if ("perro" in path_list[1]):
        real_class_indices.append(1)
print(real_class_indices)
print(len(real_class_indices))
real_class_indices = np.array(real_class_indices)
print(type(real_class_indices))

In [ ]:
fig = plt.figure(figsize=(30,30))
fig.subplots_adjust(hspace=0.1, wspace=0.1)
rows=10
cols=len(filenames)//rows if len(filenames)% 2 == 0 else len(filenames)//rows +1
folder = "data/pruebas/test_images/"
for i in range(0, len(filenames)):
    #print(filenames[i])
    your_path = filenames[i]
    path_list = your_path.split(os.sep)
    #print(path_list)
    img=mpimg.imread(folder +path_list[1])
    ax = fig.add_subplot(rows, cols, i+1)
    ax.axis('off')
    plt.imshow(img, interpolation=None)
    ax.set_title(predictions[i], fontsize=20)

In [ ]:
cm= confusion_matrix(real_class_indices,predicted_class_indices)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize = False
                          title='Confusion matrix'
                          cmap=plt.cm.Blues
                         )
    """""
    Esta funcion imprime y plotea la matris de confusion
    Normalizacionpuede ser aplicada por la configuracion de 'normalize=True'
    """""
    plt.imshow(cm,interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm=cm.astype('float')/cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print("Confusion matrix, without normalization")
    
    print(cm)
    thresh = cm.max()/2.
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                horizontalalignment="center",
                color-"white" if cm[i,j]> thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels= imagen_entrenamiento.class_indices
plot_confusion_matrix(cm,cm_plot_labels, title='Matriz fr confusion')